In [2]:
import torch
import pandas as pd
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder

In [35]:
elo = 2500
data_pandas = pd.read_csv('./data/games_cleaned_'+str(elo)+'_scrambled.csv')
# print(data_pandas)

In [298]:
def one_hot_encode_labels(labels : list[str]):
    labels_dict = {'.': 0, 'K': 1, 'Q': 2, 'B': 3, 'N': 4, 'R': 5, 'P': 6, 'k': 7, 'q': 8, 'b': 9, 'n': 10, 'r': 11, 'p': 12}
    encoded_labels = torch.zeros(len(labels), len(labels_dict))
    for i, label in enumerate(labels):
        encoded_labels[i][labels_dict[label]] = 1
    return encoded_labels.view(-1, 64*13) # will return a 1x832 which is 64x13

def encode_player_col(player : str):
    player_col = torch.Tensor([1 if player == 'white' else 0])
    return player_col

class Neuro_gambit(nn.Module):
    def __init__(self):
        super(Neuro_gambit, self).__init__()
        # self.lin_test = nn.Linear(833, 36) # input 64*13 + 1, output 36
        self.encoder = nn.Sequential(
            nn.Linear(833, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU()
        )

        self.decoder = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 36)
        )

    def forward_pandas(self, positions : list[str], player : str):
        pos_tensor = one_hot_encode_labels(positions)
        player_col = encode_player_col(player)
        input_layer = torch.cat((pos_tensor, player_col.unsqueeze(1)), dim=1)
        output = self.decoder(self.encoder(input_layer))
        return torch.split(output, [8, 8, 8, 8, 4], dim=1)

    def forward(self, x : torch.Tensor):
        output = self.decoder(self.encoder(x))
        return torch.split(output, [8, 8, 8, 8, 4], dim=1)

In [299]:
# Testing
model = Neuro_gambit()
position_columns = ['p'+str(i) for i in range(64)]
test_position = data_pandas[position_columns].iloc[0].to_list()
test_player = data_pandas['player'].iloc[0]

print('input:', test_player,test_position)

model = Neuro_gambit()
forward_test = model.forward_pandas(test_position, test_player)

print('output:', forward_test)

input: white ['.', '.', 'r', '.', '.', 'r', 'k', '.', '.', '.', '.', '.', '.', 'p', 'b', 'p', 'p', '.', 'n', '.', '.', '.', 'p', '.', '.', '.', '.', 'N', 'p', '.', '.', '.', '.', '.', '.', '.', 'P', '.', '.', '.', '.', '.', '.', '.', 'B', '.', '.', '.', 'P', 'P', 'P', '.', '.', 'P', 'P', 'P', 'R', '.', '.', '.', '.', 'R', 'K', '.']
output: (tensor([[ 0.0354,  0.0462, -0.0158, -0.0344,  0.0426, -0.0260, -0.0312,  0.0661]],
       grad_fn=<SplitWithSizesBackward0>), tensor([[-0.1150,  0.0716, -0.0906,  0.0854, -0.0111, -0.0385,  0.0046,  0.0915]],
       grad_fn=<SplitWithSizesBackward0>), tensor([[-0.1050, -0.0398,  0.0477,  0.0266, -0.0222, -0.1043,  0.0748, -0.0829]],
       grad_fn=<SplitWithSizesBackward0>), tensor([[ 0.0136,  0.0031, -0.0075, -0.0248,  0.0590, -0.0426, -0.0302,  0.0771]],
       grad_fn=<SplitWithSizesBackward0>), tensor([[-0.0580, -0.0923, -0.1206,  0.0049]],
       grad_fn=<SplitWithSizesBackward0>))


In [300]:
def encode_uci(uci : str):
    pos_rank_labels = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7}
    pos_file_labels = {'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7}
    promotion_labels = {'q': 0, 'r': 1, 'b': 2, 'n': 3}

    origin_pos = uci[:2]
    destin_pos = uci[2:4]

    org_rank = origin_pos[0]
    org_file = origin_pos[1]

    des_rank = destin_pos[0]
    des_file = destin_pos[1]

    # org pos
    encoded_org_pos_rank = torch.zeros(1, len(pos_rank_labels))
    encoded_org_pos_rank[0][pos_rank_labels[org_rank]] = 1

    encoded_org_pos_file = torch.zeros(1, len(pos_file_labels))
    encoded_org_pos_file[0][pos_file_labels[org_file]] = 1

    # des pos
    encoded_des_pos_rank = torch.zeros(1, len(pos_rank_labels))
    encoded_des_pos_rank[0][pos_rank_labels[des_rank]] = 1

    encoded_des_pos_file = torch.zeros(1, len(pos_file_labels))
    encoded_des_pos_file[0][pos_file_labels[des_file]] = 1

    encoded_prom = torch.zeros(1, len(promotion_labels)) # all zeros

    if (len(uci) == 5): # there is a promotion
        promotion = uci[4]
        for i, label in enumerate(promotion):
            encoded_prom[i][promotion_labels[label]] = 1

    return torch.cat((encoded_org_pos_rank.view(1, -1), encoded_org_pos_file.view(1, -1),
                      encoded_des_pos_rank.view(1, -1), encoded_des_pos_file.view(1, -1),
                      encoded_prom.view(1, -1)), dim=1)

def decode_uci(tensors : list[torch.Tensor]):
    pos_rank_labels = {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h'}
    pos_file_labels = {0: '1', 1: '2', 2: '3', 3: '4', 4: '5', 5: '6', 6: '7', 7: '8'}
    promotion_labels = {0: 'q', 1: 'r', 2: 'b', 3: 'n'}

    tensor_list = []
    for i in range(len(tensors)):
        tensor_list.append(tensors[i].tolist()[0])
    o_r = pos_rank_labels[tensor_list[0].index(1)]
    o_f = pos_file_labels[tensor_list[1].index(1)]
    d_r = pos_rank_labels[tensor_list[2].index(1)]
    d_f = pos_file_labels[tensor_list[3].index(1)]
    try:
        p = promotion_labels[tensor_list[4].index(1)]
    except ValueError:
        p = ''
    return o_r+o_f+d_r+d_f+p

In [291]:
# Testing 
# test2 = data_pandas['uci'].iloc[418]
test2 = data_pandas['uci'].iloc[0]
print(test2)
t = encode_uci(test2)
t_split = torch.split(t, [8, 8, 8, 8, 4], dim=1)

print(decode_uci(t_split))


c2c3
c2c3


In [222]:
# Transforming dataframe to tensor
position_columns = ['p'+str(i) for i in range(64)]

X_pos_pandas = data_pandas[position_columns]
X_col_pandas = data_pandas['player']

X_pos_values = X_pos_pandas.values
X_col_values = X_col_pandas.values
X_list = []
i = 0
for i in range(len(X_pos_values)):
    x_pos = X_pos_values[i]
    x_pos_encoded = one_hot_encode_labels(x_pos)
    x_col = X_col_values[i]
    x_col_encoded = encode_player_col(x_col)

    x = torch.cat((x_pos_encoded, x_col_encoded.unsqueeze(1)), dim=1)
    X_list.append(x)

X = torch.cat(X_list, dim=0)
print(X.shape)
torch.save(X, './data/X_tensor_'+str(elo)+'.pt')


Y_pandas = data_pandas['uci']
Y_pandas_values = Y_pandas.values
Y_list = []
for i in range(len(Y_pandas_values)):
    y_val = Y_pandas_values[i]
    y = encode_uci(y_val)
    Y_list.append(y)

Y = torch.cat(Y_list, dim=0)
print(Y.shape)
torch.save(Y, './data/Y_tensor_'+str(elo)+'.pt')

torch.Size([1266, 833])
torch.Size([1266, 36])


In [ ]:
# Training model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # using cuda
model = Neuro_gambit().to(device)

# data
X : torch.Tensor = torch.load('./data/X_tensor_'+str(elo)+'.pt').to(device)
Y : torch.Tensor = torch.load('./data/Y_tensor_'+str(elo)+'.pt').to(device)

# seperating the Y
Y1 = Y[:, :8]
Y2 = Y[:, 8:16]
Y3 = Y[:, 16:24]
Y4 = Y[:, 24:32]
Y5 = Y[:, 32:]

Y = [Y1,Y2,Y3,Y4,Y5]

# epochs, loss, and optim
learning_rate = 0.01
n_epochs = 1000000

# loss and optimizer functions from pytorch
criterion = nn.MSELoss() # MSE function
optimizer = torch.optim.SGD(params=model.parameters(), lr=learning_rate) # stochastic gradient descent function

In [308]:
# training loop
for epoch in range(n_epochs):
    # forward
    y_preds = model(X) # will output a tuple of 5 tensors

    total_loss = 0
    for i in range(len(y_preds)): # calculating the loss per tensor
        y_pred = y_preds[i]
        total_loss += criterion(y_pred, Y[i])

    # backward
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    if (epoch+1) % 5 == 0: # every 100 steps
        print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {total_loss.item():.4f}', end='\r')

KeyboardInterrupt: 

In [305]:
# sample test
import torch.nn.functional as F

with torch.no_grad():
    print(data_pandas.iloc[0]['uci'])
    x = X[0].view(1,-1)
    y = Y[0][0],Y[1][0],Y[2][0],Y[3][0],Y[4][0]
    y_preds = [t.view(-1) for t in model(x)]
    for y_iter in y:
        print(y_iter)
    for y_pred in y_preds:
        print(F.softmax(y_pred, dim=0))
    # print(criterion(y_pred,y).item())

c2c3
tensor([0., 0., 1., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 1., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 1., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 1., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0.], device='cuda:0')
tensor([0.1095, 0.1478, 0.2360, 0.1051, 0.0943, 0.1062, 0.0969, 0.1041],
       device='cuda:0')
tensor([0.1017, 0.2905, 0.0976, 0.0970, 0.1001, 0.1062, 0.1031, 0.1037],
       device='cuda:0')
tensor([0.0977, 0.1327, 0.2648, 0.1042, 0.1175, 0.0955, 0.0737, 0.1139],
       device='cuda:0')
tensor([0.1008, 0.1041, 0.2454, 0.1177, 0.1143, 0.1149, 0.0988, 0.1039],
       device='cuda:0')
tensor([0.2517, 0.2514, 0.2463, 0.2506], device='cuda:0')


In [309]:
# Save the model
torch.save(model.state_dict(), './models/neuro_gambit_'+str(elo)+'.pt')